In [1]:
import pandas as pd
import random
import uuid

# Sample data to pick from
titles = ["The Cove", "Flake", "Upside Down", "Better Together", "Banana Pancakes", 
          "Breakdown", "Sitting, Waiting, Wishing", "Good People", "Do You Remember", 
          "If I Had Eyes", "You and Your Heart"]
releases = ["Thicker Than Water", "Brushfire Fairytales", "Sing-A-Longs and Lullabies for the Film Curious George", 
            "In Between Dreams", "Sleep Through the Static", "To the Sea"]
artist_names = ["Jack Johnson", "John Mayer", "Norah Jones", "Adele", "Coldplay", "Ed Sheeran"]
years = [2000, 2001, 2005, 2006, 2008, 2010, 2011, 2013, 2015, 2018, 2020]

# Generate data
data = []
for i in range(10000):
    user_id = i
    song_id = uuid.uuid4().hex
    listen_count = random.randint(1, 20)
    title = random.choice(titles)
    release = random.choice(releases)
    artist_name = random.choice(artist_names)
    year = random.choice(years)
    data.append([user_id, song_id, listen_count, title, release, artist_name, year])

# Create DataFrame
df = pd.DataFrame(data, columns=["user_id", "song_id", "listen_count", "title", "release", "artist_name", "year"])

# Save to CSV
df.to_csv("music_data.csv", index=False)

print("Data generation complete.")


Data generation complete.


In [2]:
import pandas as pd
import random
import uuid
from sklearn.neighbors import NearestNeighbors
import numpy as np
from collections import defaultdict

In [9]:
music_data = pd.read_csv('train.csv')
recommendation_users = pd.read_csv('recommendations.csv',names = ['user_id'],header = None)

In [10]:
unique_users = music_data['user_id'].unique()
recommendation_users = random.sample(list(unique_users), 200)
df_recommendations = pd.DataFrame(recommendation_users, columns=["user_id"])
df_recommendations.to_csv("recommendations.csv", index=False)

In [11]:
user_song_matrix = music_data.pivot(index='user_id', columns='song_id', values='listen_count').fillna(0)

In [12]:
recommendation_users = pd.read_csv('recommendations.csv',names = ['user_id'],header = None)
user_song_matrix = music_data.pivot(index='user_id', columns='song_id', values='listen_count').fillna(0)

# Fit the KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(user_song_matrix)

# Create a dictionary to store recommendations
recommendations = defaultdict(list)

# Generate recommendations for each user in the recommendations.csv
for user_id in recommendation_users['user_id']:
    # Get the index of the user in the matrix
    user_index = user_song_matrix.index.get_loc(user_id)
    
    # Find the K nearest neighbors
    distances, indices = knn.kneighbors(user_song_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=11)
    
    # Get the list of neighbor indices (excluding the first one as it is the user itself)
    neighbor_indices = indices.flatten()[1:]
    
    # Get the list of songs the user has already listened to
    listened_songs = set(music_data[music_data['user_id'] == user_id]['song_id'])
    
    # Aggregate the songs listened by the neighbors
    recommended_songs = defaultdict(float)
    for neighbor_index in neighbor_indices:
        neighbor_id = user_song_matrix.index[neighbor_index]
        neighbor_songs = music_data[music_data['user_id'] == neighbor_id]
        for _, row in neighbor_songs.iterrows():
            if row['song_id'] not in listened_songs:
                recommended_songs[row['song_id']] += row['listen_count']
    
    # Get top 10 recommendations
    top_10_songs = sorted(recommended_songs, key=recommended_songs.get, reverse=True)[:10]
    
    # Add recommendations to the dictionary
    recommendations[user_id] = top_10_songs
x = pd.DataFrame(columns = ["Col1","Col2","Col3","Col4","Col5","Col6","Col7","Col8","Col9","Col10","Col11"] ).to_csv('recommendations_output.csv',index=False)
                            
# Save recommendations to CSV
with open('recommendations.csv', 'w') as f:
    for user_id, song_ids in recommendations.items():
        f.write(f"{user_id}," + ",".join(song_ids) + "\n")



KeyError: 'user_id'